# SQL-based database

Defines an sqlite database, built from the json-based data dumps provided at 'https://github.com/adactio/TheSession-data'.

Tables are included for:

- Users
- Tunes
- Settings
- Aliases
- Recordings
- Recorded Tunes
- Events
- Sessions

Fields are added to the database from additional scraped directly from thesession.org, using the sites api, and from information consolidated by this codebase.

I plan to extend the database class with a variety of pythonic seach functions wrapping SQL queries for specific needs from other parts of the code.

In [1]:
from thesessionDB import thesessionDB

db = thesessionDB(delete=True,download=False)

db.defineTables()
#db.populateTablesFromJSON(Ntest=1000)
db.populateTablesFromJSON()

db.cur.execute('SELECT * FROM users LIMIT 5')
q = db.cur.fetchall()  # fetch all the results of the query

print ''
print 'Users:'
for x in q:
    print x

db.cur.execute('SELECT * FROM tunes LIMIT 5')
q = db.cur.fetchall()  # fetch all the results of the query

print ''
print 'Tunes:'
for x in q:
    print x

db.cur.execute('SELECT setting_id, tune_id, date, user FROM settings LIMIT 5')
q = db.cur.fetchall()  # fetch all the results of the query

print ''
print 'Settings:'
for x in q:
    print x

db.cur.execute('SELECT * FROM aliases LIMIT 5')
q = db.cur.fetchall()  # fetch all the results of the query

print ''
print 'Aliases:'
for x in q:
    print x

db.cur.execute('SELECT * FROM artists LIMIT 5')
q = db.cur.fetchall()  # fetch all the results of the query

print ''
print 'Artists:'
for x in q:
    print x

db.cur.execute('SELECT * FROM recordings LIMIT 5')
q = db.cur.fetchall()  # fetch all the results of the query

print ''
print 'Recordings:'
for x in q:
    print x

db.cur.execute('SELECT * FROM recorded_tunes LIMIT 5')
q = db.cur.fetchall()  # fetch all the results of the query

print ''
print 'Recorded tunes:'
for x in q:
    print x


Transferring tune settings from JSON to SQL
Transferring tune aliases from JSON to SQL
Transferring recordings from JSON to SQL
Transferring events from JSON to SQL
Transferring sessions from JSON to SQL

Users:
(u'iliketurtles',)
(u'Daniel Quayle',)
(u'DonaldK',)
(u'Charles Mackenzie',)
(u'Micha\u0142 Zaj\u0105czkowski',)

Tunes:
(u'11931', u"'G Iomain Nan Gamhna", u'slip jig', u'9/8', u'Gmajor')
(u'15326', u"'S Ann An \xccle", u'strathspey', u'4/4', u'Gmajor')
(u'14625', u"'S Daor An Tabac", u'reel', u'4/4', u'Bminor')
(u'13684', u'100 Watt Reels', u'reel', u'4/4', u'Ddorian')
(u'7078', u'10th Bat Crossing Rhine', u'jig', u'6/8', u'Amixolydian')

Settings:
(u'11931', u'11931', u'2012-05-17 07:49:26', u'iliketurtles')
(u'28560', u'15326', u'2016-03-31 15:34:45', u'Daniel Quayle')
(u'28582', u'15326', u'2016-04-03 09:15:08', u'DonaldK')
(u'26955', u'14625', u'2015-07-31 02:47:47', u'Charles Mackenzie')
(u'24291', u'13684', u'2014-06-14 13:57:17', u'Micha\u0142 Zaj\u0105czkowski')

Alia

In [126]:
#import pyabc
from pyabc import *
import json
import numpy as np

import os
import sqlite3  # this is the module that binds to SQLite
import pandas as pd  # you'll see why we can use this later

from progress import progress

https://thesession.org/api

Formats
JSON	?format=json
XML	?format=xml
RSS	?format=rss
Endpoints
The following lists are available in multiple formats e.g. /tunes/new?format=json or /recordings/search?q=altan&format=rss

By default, 10 items will be returned in a list. You can request up to 50 items by appending &perpage= e.g. /tunes/new?format=json&perpage=35

If you request an individual item, e.g. /tunes/27?format=xml, you will get back the details for that item and any comments that have been posted to it.

In [74]:
# setup database

DBFILE = 'thesession.db'  # this will be our database
BASEDIR = os.getcwd() + '/sqlite3'  # os.path.abspath(os.path.dirname(__file__))
DBPATH = os.path.join(BASEDIR, DBFILE)

# we may need to delete the existing file first
if os.path.exists(DBPATH):
    os.remove(DBPATH)

# open a connection to the database for this tutorial
conn = sqlite3.connect(DBPATH)

# get a cursor to the database
cur = conn.cursor()

In [9]:
def get_thesession_jsons():
    import os, json
    #if not os.path.isfile("tunes.json"):
    import sys, urllib
    for f in ['tunes','recodings','aliases','events','sessions']:
        url = 'https://raw.githubusercontent.com/adactio/TheSession-data/master/json/'+f+'.json'
        print("Downloading "+f+" database from %s..." % url)
        try:
            urllib.urlretrieve(url, 'json/'+f+'.json')
        except AttributeError:
            import urllib.request
            urllib.request.urlretrieve(url, 'json/'+f+'.json')
    #return json.loads(open('tunes.json', 'rb').read().decode('utf8'))
    
#get_thesession_jsons()

In [10]:
json_tunes = json.loads(open('json/tunes.json', 'rb').read().decode('utf8'))
json_recordings = json.loads(open('json/recordings.json', 'rb').read().decode('utf8'))
json_aliases = json.loads(open('json/aliases.json', 'rb').read().decode('utf8'))
json_events = json.loads(open('json/events.json', 'rb').read().decode('utf8'))
json_sessions = json.loads(open('json/sessions.json', 'rb').read().decode('utf8'))

In [14]:
# unicode
print json_tunes[1]['name']

'S Ann An Ìle


In [15]:
n_test = 10

In [78]:
json_tunes[41]

{u'abc': u'| d2c B2A | F2A DEF | G2G GFG | BAG ABA |\r\n| d2c B2A | F2A DEF | G2G ABc| dAF D3 :|\r\n| A,3 D3 | FEF FEF | GFG GFG | BAG ABA |\r\n| A,3 D3 | FEF FEF | GFG GFG | BGE D3 :|',
 u'date': u'2016-07-03 19:56:05',
 u'meter': u'6/8',
 u'mode': u'Dmajor',
 u'name': u'6/8 De Rap\xeddes',
 u'setting': u'28943',
 u'tune': u'15458',
 u'type': u'jig',
 u'username': u'Jesse'}

In [187]:
# From tunes.json

# users 
try:
    cur.execute('DROP TABLE users')
    #conn.commit()
except:
    pass

cur.execute('''CREATE TABLE users (
    user TEXT PRIMARY KEY
)''')
conn.commit()

# tunes
try:
    cur.execute('DROP TABLE tunes')
    #conn.commit()
except:
    pass

cur.execute('''CREATE TABLE tunes (
    tune_id TEXT PRIMARY KEY,
    name TEXT NOT NULL,
    rhythm TEXT NOT NULL,
    meter TEXT NOT NULL,
    mode TEXT NOT NULL
)''')
conn.commit()

# settings
try:
    cur.execute('DROP TABLE settings')
    #conn.commit()
except:
    pass

cur.execute('''CREATE TABLE settings (
    setting_id TEXT PRIMARY KEY,
    tune_id TEXT NOT NULL REFERENCES tunes ON UPDATE CASCADE ON DELETE CASCADE,
    abc  TEXT,
    date TEXT NOT NULL,
    user TEXT NOT NULL REFERENCES users on UPDATE CASCADE ON DELETE CASCADE,
    name TEXT NOT NULL,
    rhythm TEXT NOT NULL,
    meter TEXT NOT NULL,
    mode TEXT NOT NULL,
    bars INTEGER,
    structure TEXT
)''')
conn.commit()

In [188]:
# Insert entries from tunes.json
N = 1000
for i,jj in enumerate(json_tunes[:N]): 
    
    # Fill missing with None
    keys = ['setting','tune','abc','date','username','name','type','meter','mode']
    j = {}   
    for x in keys:
        try:
            j.update({x:jj[x]})
        except:
            j.update({x:None})
    
    # Check if user is new, if so add to table
    cur.execute('SELECT * FROM users WHERE (user=?)', [j['username']])
    entry = cur.fetchone()
    if entry is None:
        cur.execute('INSERT INTO users VALUES (?)', [j['username']])
        
    # Check if tune is new, if so add to table
    cur.execute('SELECT * FROM tunes WHERE (tune_id=?)', [j['tune']])
    entry = cur.fetchone()
    if entry is None:
        row = [j['tune'],j['name'],j['type'],j['meter'],j['mode']]
        cur.execute('INSERT INTO tunes VALUES (?,?,?,?,?)', row)

    row = [j['setting'],j['tune'],j['abc'],j['date'],j['username'],j['name'],j['type'],j['meter'],j['mode']]
    cur.execute('INSERT INTO settings VALUES (?,?,?,?,?,?,?,?,?,NULL,NULL)',row)
    progress(i+1,N,'Transfering tune settings from JSON to SQL.')
    
conn.commit()  # you must commit for it to become permanent


In [189]:
cur.execute('SELECT * FROM users')
cur.fetchall()  # fetch all the results of the query

[(u'iliketurtles',),
 (u'Daniel Quayle',),
 (u'DonaldK',),
 (u'Charles Mackenzie',),
 (u'Micha\u0142 Zaj\u0105czkowski',),
 (u'gaitazampogna_32',),
 (u'ceolachan',),
 (u'fluther',),
 (u'JACKB',),
 (u'pcd',),
 (u'JWiseman',),
 (u'abush',),
 (u'Will Harmon',),
 (u'pbsinclair42',),
 (u'Edgar Bolton',),
 (u'Frulator',),
 (u'Rachael',),
 (u'callison',),
 (u'ukejosh',),
 (u'flamin fiddler',),
 (u'Ellie Goodman',),
 (u'gian marco',),
 (u"Juste O'Curran-Twomey",),
 (u'Dr. T\xf8m',),
 (u'jdicarlo',),
 (u'JosephC',),
 (u'bdh',),
 (u'Mr G. Cunningham',),
 (u'Nigel Gatherer',),
 (u'multimentalist',),
 (u'lerinrin',),
 (u'ChrisJCorrigan',),
 (u'Jesse',),
 (u"Mix O'Lydian",),
 (u'snowyowl',),
 (u'jchillerup',),
 (u'jaychoons',),
 (u'zoukboy_2000',),
 (u'malcombpiper',),
 (u'birlibirdie',),
 (u'whistlefanatic',),
 (u'Crackpot',),
 (u'secretsquirrel',),
 (u'creathana',),
 (u'niall_kenny',),
 (u'Moxhe',),
 (u'MarcusDisessa',),
 (u'Mikethebook',),
 (u'Sl\xe1ine',),
 (u'aidriano',),
 (u'Thistledowne',),


In [190]:
cur.execute('SELECT * FROM tunes')
cur.fetchall()  # fetch all the results of the query

[(u'11931', u"'G Iomain Nan Gamhna", u'slip jig', u'9/8', u'Gmajor'),
 (u'15326', u"'S Ann An \xccle", u'strathspey', u'4/4', u'Gmajor'),
 (u'14625', u"'S Daor An Tabac", u'reel', u'4/4', u'Bminor'),
 (u'13684', u'100 Watt Reels', u'reel', u'4/4', u'Ddorian'),
 (u'7078', u'10th Bat Crossing Rhine', u'jig', u'6/8', u'Amixolydian'),
 (u'15800', u'12 Weeks And A Day', u'reel', u'4/4', u'Bminor'),
 (u'16173', u'15th Scottish Division', u'waltz', u'3/4', u'Dmajor'),
 (u'9294', u'18-6 Two Step', u'jig', u'6/8', u'Gmajor'),
 (u'16123', u'1921', u'reel', u'4/4', u'Dmajor'),
 (u'935', u'1st August', u'reel', u'4/4', u'Edorian'),
 (u'16424', u'2-Rivisjenkka', u'hornpipe', u'4/4', u'Eminor'),
 (u'10999', u'249 To Wigton', u'jig', u'6/8', u'Dminor'),
 (u'2711', u'250 To Vigo', u'reel', u'4/4', u'Bminor'),
 (u'15717', u'28th Of January, The', u'reel', u'4/4', u'Adorian'),
 (u'10543', u'2D Or Not 2D', u'mazurka', u'3/4', u'Dmajor'),
 (u'14735', u'3 AM In Edradour', u'reel', u'4/4', u'Adorian'),
 (u'

In [191]:
cur.execute('SELECT setting_id, tune_id, date, user FROM settings')
cur.fetchall()  # fetch all the results of the query

[(u'11931', u'11931', u'2012-05-17 07:49:26', u'iliketurtles'),
 (u'28560', u'15326', u'2016-03-31 15:34:45', u'Daniel Quayle'),
 (u'28582', u'15326', u'2016-04-03 09:15:08', u'DonaldK'),
 (u'26955', u'14625', u'2015-07-31 02:47:47', u'Charles Mackenzie'),
 (u'24291',
  u'13684',
  u'2014-06-14 13:57:17',
  u'Micha\u0142 Zaj\u0105czkowski'),
 (u'7078', u'7078', u'2007-04-15 06:56:09', u'gaitazampogna_32'),
 (u'18649', u'7078', u'2007-04-18 13:25:41', u'ceolachan'),
 (u'18650', u'7078', u'2007-04-18 14:05:40', u'ceolachan'),
 (u'29725', u'15800', u'2017-01-27 19:19:27', u'fluther'),
 (u'30543', u'16173', u'2017-06-22 14:46:14', u'JACKB'),
 (u'30544', u'16173', u'2017-06-22 14:47:19', u'JACKB'),
 (u'31268', u'16173', u'2017-11-08 03:13:15', u'JACKB'),
 (u'9294', u'9294', u'2009-01-25 05:35:16', u'pcd'),
 (u'30395', u'16123', u'2017-05-27 11:15:54', u'JWiseman'),
 (u'935', u'935', u'2002-08-29 18:36:22', u'abush'),
 (u'14126', u'935', u'2002-09-01 06:59:01', u'Will Harmon'),
 (u'31116', u

In [179]:
# from aliases.json

# Create aliases table
try:
    cur.execute('DROP TABLE aliases')
    #conn.commit()
except:
    pass

cur.execute('''CREATE TABLE aliases (
    alias_id INTEGER PRIMARY KEY,
    alias TEXT NOT NULL,
    name TEXT NOT NULL,
    tune_id NOT NULL REFERENCES tunes ON UPDATE CASCADE ON DELETE CASCADE
)''')
conn.commit()

In [180]:
# insert entries from aliases.json
N = 1000
cur.execute('SELECT MAX(alias_id) from aliases')
try:
    startid = int(cur.fetchone()[0])+1
except:
    startid = 0
        
for i,j in enumerate(json_aliases[:N]):
    keys = ['alias','name','tune_id']
    jj = {}   
    for x in keys:
        try:
            jj.update({x:j[x]})
        except:
            jj.update({x:None})
    
    row = [startid+i,jj['alias'],jj['name'],jj['tune_id']]
    cur.execute('INSERT INTO aliases VALUES (?,?,?,?)',row)
    progress(i+1,N,'Transferrring aliases from JSON to SQL.')
conn.commit()

In [178]:
cur.execute('SELECT MAX(alias_id) from aliases')
startid = int(cur.fetchone()[0])+1
startid

1000

In [181]:
cur.execute('SELECT * FROM aliases')
cur.fetchall()  # fetch all the results of the query

[(0, u'Cooleys', u"Cooley's", u'1'),
 (1, u'Joe Cooley', u"Cooley's", u'1'),
 (2, u"Joe Cooley's", u"Cooley's", u'1'),
 (3, u"Joe Cooley's Fancy", u"Cooley's", u'1'),
 (4, u'Joe Cooleys', u"Cooley's", u'1'),
 (5, u"Luttrell's Pass", u"Cooley's", u'1'),
 (6, u'Put The Cake In The Dresser', u"Cooley's", u'1'),
 (7, u'Put The Cake On The Dresser', u"Cooley's", u'1'),
 (8, u'Tulla, The', u"Cooley's", u'1'),
 (9, u'Box Will Hold No More, The', u'Bucks Of Oranmore, The', u'2'),
 (10, u'Buckets Of More And More', u'Bucks Of Oranmore, The', u'2'),
 (11, u'Bucks Of Aranmore, The', u'Bucks Of Oranmore, The', u'2'),
 (12, u'Bucks Of Arranmore, The', u'Bucks Of Oranmore, The', u'2'),
 (13, u'Bucks Of Bohermore, The', u'Bucks Of Oranmore, The', u'2'),
 (14, u'Bucks Of Clare, The', u'Bucks Of Oranmore, The', u'2'),
 (15, u'Bucks Of Oran Mor, The', u'Bucks Of Oranmore, The', u'2'),
 (16, u'Bucks, The', u'Bucks Of Oranmore, The', u'2'),
 (17, u'Hearty Bucks Of Oranmore, The', u'Bucks Of Oranmore, The'

In [170]:
# from recordings.json

#artists
try:
    cur.execute('DROP TABLE artists')
    #conn.commit()
except:
    pass

cur.execute('''CREATE TABLE artists (
    artist TEXT PRIMARY KEY
)''')
conn.commit()

# recordings  (albums)
try:
    cur.execute('DROP TABLE recordings')
    #conn.commit()
except:
    pass

cur.execute('''CREATE TABLE recordings (
    recording_id TEXT PRIMARY KEY,
    recording_name TEXT NOT NULL,
    artist TEXT REFERENCES artists ON UPDATE CASCADE ON DELETE CASCADE
)''')
conn.commit()

# recorded tunes
try:
    cur.execute('DROP TABLE recorded_tunes')
    #conn.commit()
except:
    pass

cur.execute('''CREATE TABLE recorded_tunes (
    id TEXT PRIMARY KEY,
    recording_id TEXT REFERENCES recordings ON UPDATE CASCADE ON DELETE CASCADE,
    recording_name TEXT NOT NULL,
    artist TEXT REFERENCES artists ON UPDATE CASCADE ON DELETE CASCADE,
    track INTEGER NOT NULL,
    number INTEGER NOT NULL,
    tune_name TEXT, 
    tune_id NOT NULL REFERENCES tunes ON UPDATE CASCADE ON DELETE CASCADE
    
)''')
conn.commit()

In [171]:
# Insert entries from recordings.json
N = 1000
for i,j in enumerate(json_recordings[:N]): 
    
    keys = ['id','recording','artist','track','number','tune','tune_id']
    jj = {}   
    for x in keys:
        try:
            jj.update({x:j[x]})
        except:
            jj.update({x:None})
    
    # Check if user is new, if so add to table
    cur.execute('SELECT * FROM artists WHERE (artist=?)', [jj['artist']])
    entry = cur.fetchone()
    if entry is None:
        cur.execute('INSERT INTO artists VALUES (?)', [jj['artist']])
        
    # Check if tune is new, if so add to table
    cur.execute('SELECT * FROM recordings WHERE (recording_id=?)', [jj['id']])
    entry = cur.fetchone()
    if entry is None:
        row = [j['id'],j['recording'],j['artist']]
        cur.execute('INSERT INTO recordings VALUES (?,?,?)', row)

    row = [i,jj['id'],jj['recording'],jj['artist'],int(jj['track']),int(jj['number']),jj['tune'],jj['tune_id']]
    cur.execute('INSERT INTO recorded_tunes VALUES (?,?,?,?,?,?,?,?)',row)
    progress(i+1,N,'Transferring recordings from JSON to SQL.')
    
conn.commit()  # you must commit for it to become permanent
    #cur.rowcount  # tells you how many rows written, sometimes, it's quirky

In [167]:
cur.execute('SELECT * FROM recordings')
cur.fetchall()  # fetch all the results of the query

[(u'3720', u'Cast A Bell', u'1651'),
 (u'3736', u'Irish Folk Songs', u'1691'),
 (u'3266', u'Until The Cows Come Home', u'2duos'),
 (u'1871', u'Byth Adra', u'4 Yn Y Bar'),
 (u'3984', u'Go Forth', u'422'),
 (u'2631', u'Major Third', u'422'),
 (u'2630', u'New Numbers', u'422'),
 (u'2394', u'One', u'422'),
 (u'3216', u'20:20 Manchester', u'4Square'),
 (u'3885', u'ChronicLes', u'4Square'),
 (u'3606', u'Buttered Side Down', u'5 Second Rule'),
 (u'3808', u'A Crowd Of Bold Sharemen', u'A Crowd Of Bold Sharemen'),
 (u'3598', u'Uilleann Piping, Another Generation', u'A New Dawn'),
 (u'4031', u'Caravan', u'AAMOS'),
 (u'5362',
  u'Traditional Tunes from Newfoundland & Labrador',
  u'Aaron Collis and Emilia Bartellas'),
 (u'3719', u'Castles, Kirks And Caves', u'Abby Newton'),
 (u'4000', u'Crossing To Scotland', u'Abby Newton'),
 (u'4129', u'Y Bwced Perffaith: The Perfect Bucket', u'Aberjaber'),
 (u'1741', u'The Bardic Moods Of Music', u'Abhan Trireach'),
 (u'2598', u'Samhradh', u'Abisko'),
 (u'3948

In [152]:
# from events.json
try:
    cur.execute('DROP TABLE events')
    #conn.commit()
except:
    pass

cur.execute('''CREATE TABLE events (
    event_id TEXT PRIMARY KEY,
    address TEXT,
    area TEXT,
    country TEXT,
    dtend TEXT,
    dtstart TEXT,
    event_name TEXT,
    latitude TEXT,
    logitude TEXT,
    town TEXT,
    venue TEXT
)''')
conn.commit()

In [129]:
progress?

In [153]:
N = 1000
for i,j in enumerate(json_events[:N]):    

    keys = ['id','address','area','country','dtend','dtstart','event','latitude','longitude','town','venue']
    row = [ None for x in range(len(keys))]
    
    for x in  range(len(keys)):
        try:
            row[x] = j[keys[x]]
        except:
            pass
    #row = [j['id'],j['address'],j['area'],j['country'],j['dtend'],j['dtstart'],j['event'],
    #      j['latitude'],j['longitude'],j['town'],j['venue']]
    cur.execute('INSERT INTO events VALUES (?,?,?,?,?,?,?,?,?,?,?)',row)
    conn.commit()  # you must commit for it to become permanent
    #cur.rowcount  # tells you how many rows written, sometimes, it's quirky
    progress(i+1,N,'Transfering events from JSON to SQL.')

In [131]:
cur.execute('SELECT * FROM events')
cur.fetchall()  # fetch all the results of the query

[(u'11',
  u'226 Water Street',
  u'Massachusetts',
  u'USA',
  u'2006-06-07 12:00:00',
  u'2006-06-07 09:30:00',
  u'Colm Gannon, Sean Mckeon And John Blake',
  u'42.24073792',
  u'-71.00814819',
  u'Quincy',
  u'The Goalpost'),
 (u'5',
  u'Causeway Link',
  u'Clare',
  u'Ireland',
  u'2006-06-09 00:00:00',
  u'2006-06-09 08:00:00',
  u"Brid O'Donohue",
  u'52.84102631',
  u'-8.99745941',
  u'Ennis',
  u'Gl\xf3r'),
 (u'6',
  u'',
  u'Victoria',
  u'Australia',
  u'2006-06-12 15:00:00',
  u'2006-06-09 19:00:00',
  u'National Celtic Festival',
  u'-38.11458588',
  u'144.65211487',
  u'Portarlington',
  u'Various Venues'),
 (u'7',
  u'200 New Boston Drive',
  u'Massachusetts',
  u'USA',
  u'2006-06-11 00:00:00',
  u'2006-06-09 19:00:00',
  u'The Irish Connections Festival',
  u'42.15058899',
  u'-71.11404419',
  u'Canton',
  u'Irish Cultural Centre'),
 (u'19',
  u'',
  u'Kentucky',
  u'USA',
  u'2006-06-11 06:00:00',
  u'2006-06-10 11:00:00',
  u'Louisville Irish Fest',
  u'38.25266647',

In [144]:
# from sessions.json
try:
    cur.execute('DROP TABLE sessions')
    #conn.commit()
except:
    pass

cur.execute('''CREATE TABLE sessions (
    session_id TEXT PRIMARY KEY,
    address TEXT,
    area TEXT,
    country TEXT,
    date TEXT,
    latitude TEXT,
    logitude TEXT,
    name TEXT,
    town TEXT
)''')
conn.commit()

In [67]:
json_sessions[1000]

{u'address': u'Plunkett St',
 u'area': u'Kerry',
 u'country': u'Ireland',
 u'date': u'2007-06-20 07:19:29',
 u'id': u'1646',
 u'latitude': u'52.05859756',
 u'longitude': u'-9.50825500',
 u'name': u'Courtneys Bar',
 u'town': u'Killarney'}

In [145]:
N = 1000
for i,j in enumerate(json_sessions[:N]):    

    keys = ['id','address','area','country','date','latitude','longitude','name','town']
    row = [ None for x in range(len(keys))]
    
    for x in  range(len(keys)):
        try:
            row[x] = j[keys[x]]
        except:
            pass
       
    #row = [j['id'],j['address'],j['area'],j['country'],j['date'],
    #      j['latitude'],j['longitude'],j['name'],j['town']]
    
    cur.execute('INSERT INTO sessions VALUES (?,?,?,?,?,?,?,?,?)',row)
    #cur.rowcount  # tells you how many rows written, sometimes, it's quirky
    progress(i+1,N,'Transfering sessions from JSON to SQL.')
conn.commit()  # you must commit for it to become permanent

In [146]:
cur.execute('SELECT * FROM sessions')
cur.fetchall()  # fetch all the results of the query

[(u'3331',
  u'',
  u'Buenos Aires',
  u'Argentina',
  u'2013-03-06 21:02:44',
  u'-34.56250000',
  u'-58.45833206',
  u'A Confirmar',
  u'Belgrano'),
 (u'3948',
  u'Zeballos 1165',
  u'Santa Fe',
  u'Argentina',
  u'2015-03-31 19:16:32',
  u'-32.95478058',
  u'-60.64122772',
  u'L\xfacuma Bar',
  u'Rosario'),
 (u'841',
  u'Garema Place',
  u'ACT',
  u'Australia',
  u'2004-10-11 17:11:03',
  u'-35.27838898',
  u'149.13209534',
  u"King O'Malley's",
  u'Canberra'),
 (u'2391',
  u'',
  u'ACT',
  u'Australia',
  u'2009-08-25 18:33:06',
  u'-35.28200150',
  u'149.12867737',
  u'Monaro Folk Society',
  u'Canberra'),
 (u'3984',
  u'Wilson Drive',
  u'New South Wales',
  u'Australia',
  u'2015-05-27 08:29:57',
  u'-34.30689240',
  u'150.52127075',
  u'Community Hall',
  u'Balmoral Village'),
 (u'2021',
  u'1 Church Street',
  u'New South Wales',
  u'Australia',
  u'2008-08-12 05:43:08',
  u'-30.45131683',
  u'152.89714050',
  u'Church Street Cafe',
  u'Bellingen'),
 (u'1987',
  u'Brisbane Str

TypeError: 'NoneType' object is not iterable